<div style="background:#F5F7FA; height:100px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Want to do more?</span><span style="border: 1px solid #3d70b2;padding: 15px;float:right;margin-right:40px; color:#3d70b2; "><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
<span style="color:#5A6872;"> Try out this notebook with your free trial of IBM Watson Studio.</span>
</div>

<div class="pull-left"><left><img style="float: right;" src="http://developer.ibm.com/clouddataservices/wp-content/uploads/sites/85/2018/01/ibm-cloud-object-storage-logo-small.png" width="100" margin=50></left></div>
<div style="text-align:center">
<b>IBM Cloud SQL Query</b> is IBM's <b>serverless SQL</b> service on data in <b>Cloud Object Storage</b>. It allows to run ANSI SQL on Parquet, CSV and JSON data sets. It is based on Apache Spark SQL as the query engine in the background. This means you do NOT have to provision any Apache Spark instance or service. A simple Python client (like the IBM Watson Studio Notebook) is sufficient.<br><br></div>
This notebook is a starter to use the SQL Query API in order to run SQL statements in a programmatic way. It uses the <a href="https://github.com/IBM-Cloud/sql-query-clients/tree/master/Python" target="_blank" rel="noopener noreferrer">ibmcloudsql</a> Python library for this purpose. The notebook also demonstrates how you can combine SQL Query with visualization libraries such as **PixieDust**. The notebook has been verified to work with Python 3.5. As mentioned above it does not require a Spark service bound to the notebook.

## Table of contents
1. [Setup libraries](#setup)<br>
2. [Configure SQL Query](#configure)<br>
    2.1 [Using the project bucket](#projectbucket)<br>
    2.2 [Setting SQL Query parameters](#parameters)<br>
3. [Your SQL](#sql)<br>
4. [Run the SQL](#run)<br>
    4.1 [Low level SQL job submission](#lowlevel)<br>
5. [List recent SQL submissions](#joblist)<br>
5. [Next steps](#next)<br>

### <a id="setup"></a> 1. Setup libraries

Run the following cell at least once in your notebook environment in order to install required packages, such as the SQL Query client library:

In [1]:
!pip -q install ibmcloudsql
!pip -q install sqlparse

In [2]:
import ibmcloudsql
from pixiedust.display import *
import pandas as pd
targeturl=''

Pixiedust database opened successfully
Table VERSION_TRACKER created successfully


Pixiedust runtime updated. Please restart kernel
Table USER_PREFERENCES created successfully
Table service_connections created successfully


### <a id="configure"></a> 2. Configure SQL Query
1. You need an **API key** for an IBM cloud identity that has access to your Cloud Object Storage bucket for writing SQL results and to your Cloud SQL Query instance. To create API keys log on to the IBM Cloud console and go to <a href="https://console.bluemix.net/iam/#/apikeys" target="_blank">Manage->Security->Platform API Keys</a>, press the `Create` button, give the key a custom name and press `Create`. In the next dialog click `Show` and copy the key to your clipboard and paste it below in this notebook.
2. You need the **instance CRN** for the Cloud SQL Query instance. You can find it in the <a href="https://console.bluemix.net/dashboard/apps" target="_blank">IBM Cloud console dashboard</a>. Make sure you have `All Resources` selected as resource group. In the section `Services` you can see your instances of SQL Query and Cloud Object Storage. Select the instance of SQL Query that you want to use. In the SQL Query dashboard page that opens up you find a section titled **REST API** with a button labelled **Instance CRN**. Click the button to copy the CRN into your clipboard and paste it here in the notebook. If you do not have a SQL Query instance created yet, please <a href="https://console.bluemix.net/catalog/services/sql-query" target="_blank">create one</a> first.
3. You need to specify the location on Cloud Object Storage where your **query results** should be written. This comprises three parts of information that you can find in the Cloud Object Storage UI for your instance in IBM Cloud console. You need to provide it as a **URL** using the format `cos://<endpoint>/<bucket>/[<prefix>]`. You have the option to use the cloud object storage **bucket that is associated with your project**. In this case please execute the following section before you proceed.  
<br/>
You can find further background in the SQL Query <a href="https://console.bluemix.net/docs/services/sql-query/getting-started.html#getting-started-tutorial" target="_blank">documentation</a>.

#### <a id="projectbucket"></a> 2.1 Using the project bucket
You need to follow the instructions in this section **only** when you want to write your SQL query results to the bucket that has been created for the project where you have created this notebook. In any other case proceed directly with section **2.2**.
<br><br>
__Insert project token__:  
Click the `More` option in the toolbar above (the three stacked dots) and select `Insert project token`.
 * If you haven't created an access token for this project before you will see a dialog that asks you to create one first. Follow the link to open your project settings, scroll down to `Access tokens` and click `New token`. Give the token a custom name and make sure you select `Editor` as `Access role for project`. After you created your access token you can come back to this notebook, select the empty cell below and again select `Insert project token` from the toolbar at the top.
[//]: # 
This will add a new cell at the top of your notebook with content that looks like this:
```
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='<some id>', project_access_token='<some access token>')
pc = project.project_context
```
Leave that cell content as inserted and run the cell. Then then proceed with the following cell below:

In [4]:
cos_bucket = project.get_metadata()['entity']['storage']['properties']
targeturl="cos://" + cos_bucket['bucket_region'] + "/" + cos_bucket['bucket_name'] + "/"

#### <a id="parameters"></a> 2.2 Setting SQL Query parameter

In [5]:
import getpass
apikey=getpass.getpass('Enter IBM Cloud API Key (leave empty to use previous one): ') or apikey
instnacecrn=input('Enter SQL Query Instance CRN (leave empty to use previous one): ') or instnacecrn
if targeturl == '':
    targeturl=input('Enter target URL for SQL results: ')
else:
    targeturl=input('Enter target URL for SQL results (leave empty to use ' + targeturl + '): ') or targeturl
sqlClient = ibmcloudsql.SQLQuery(apikey, instnacecrn, targeturl, client_info='SQL Query Starter Notebook')
sqlClient.logon()
print('\nYour SQL Query web console link:\n')
sqlClient.sql_ui_link()

Enter IBM Cloud API Key (leave empty to use previous one): ········
Enter SQL Query Instance CRN (leave empty to use previous one): crn%3Av1%3Abluemix%3Apublic%3Asql-query%3Aus-south%3Aa%2Fd86af7367f70fba4f306d3c19c938f2f%3Ad1b2c005-e3d8-48c0-9247-e9726a7ed510%3A%3A
Enter target URL for SQL results (leave empty to use cos://us-geo/samplenotebooksbaa5120fe81044f48b0f7ed580cc5554/): cos://us-south/sqltempregional/

Your SQL Query web console link:

https://sql.ng.bluemix.net/sqlquery/?instance_crn=crn:v1:bluemix:public:sql-query:us-south:a/d86af7367f70fba4f306d3c19c938f2f:d1b2c005-e3d8-48c0-9247-e9726a7ed510::


### <a id="sql"></a> 3. Your SQL
If you want to author your own SQL query we recommend you to do this in the interactive SQL Query web console (**link above**) of you SQL Query service instance.

In [6]:
import sqlparse
from pygments import highlight
from pygments.lexers import get_lexer_by_name
from pygments.formatters import HtmlFormatter, Terminal256Formatter

sql=input('Enter your SQL statement (leave empty to use a simple sample SQL)')
if sql == '':
    sql='SELECT o.OrderID, c.CompanyName, e.FirstName, e.LastName FROM cos://us-geo/sql/orders.parquet STORED AS PARQUET o, cos://us-geo/sql/employees.parquet STORED AS PARQUET e, cos://us-geo/sql/customers.parquet STORED AS PARQUET c WHERE e.EmployeeID = o.EmployeeID AND c.CustomerID = o.CustomerID AND o.ShippedDate > o.RequiredDate AND o.OrderDate > "1998-01-01" ORDER BY c.CompanyName'
formatted_sql = sqlparse.format(sql, reindent=True, indent_tabs=True, keyword_case='upper')
lexer = get_lexer_by_name("sql", stripall=True)
formatter = Terminal256Formatter(style='tango')
result = highlight(formatted_sql, lexer, formatter)
from IPython.core.display import display, HTML
print('\nYour SQL statement is:\n')
print(result)

Enter your SQL statement (leave empty to use a simple sample SQL)

Your SQL statement is:

SELECT o.OrderID,
	c.CompanyName,
	e.FirstName,
	e.LastName
FROM cos://us-geo/SQL/orders.parquet STORED AS PARQUET o,
	cos://us-geo/SQL/employees.parquet STORED AS PARQUET e,
	cos://us-geo/SQL/customers.parquet STORED AS PARQUET c
WHERE e.EmployeeID = o.EmployeeID
		AND c.CustomerID = o.CustomerID
		AND o.ShippedDate > o.RequiredDate
		AND o.OrderDate > "1998-01-01"
ORDER BY c.CompanyName



### <a id="run"></a> 4. Run the SQL
The following cell submits the SQL statement and waits for it to finish before printing a sample of the result set.

In [7]:
result_df = sqlClient.run_sql(sql)
if isinstance(result_df, str):
    print(result_df)

In [8]:
result_df.head(10)

,OrderID,CompanyName,FirstName,LastName
0,10924,Berglunds snabbköp,Janet,Leverling
1,11058,Blauer See Delikatessen,Anne,Dodsworth
2,10827,Bon app',Nancy,Davolio
3,11076,Bon app',Margaret,Peacock
4,11045,Bottom-Dollar Markets,Michael,Suyama
5,10970,Bólido Comidas preparadas,Anne,Dodsworth
6,11054,Cactus Comidas para llevar,Laura,Callahan
7,11008,Ernst Handel,Robert,King
8,11072,Ernst Handel,Margaret,Peacock
9,10816,Great Lakes Food Market,Margaret,Peacock


In [10]:
from pixiedust.display import *
display(result_df)

#### <a id="lowlevel"></a> 4.1 Low level SQL job submission
In the following we run the same SQL again. But this time we use asynchronous submission mechanism and the status check method.

In [11]:
sqlClient.logon()
jobId = sqlClient.submit_sql(sql)
print("SQL query submitted and running in the background. jobId = " + jobId)

SQL query submitted and running in the background. jobId = 6b085cc0-c01d-4e96-8c72-f7ce13033d3f


In [12]:
print("Job status for " + jobId + ": " + sqlClient.get_job(jobId)['status'])

Job status for 6b085cc0-c01d-4e96-8c72-f7ce13033d3f: running


You canuse the `wait_for_job()` method to block until your job has finished:

In [13]:
job_status = sqlClient.wait_for_job(jobId)
print("Job " + jobId + " terminated with status: " + job_status)
if job_status == 'failed':
    details = sqlClient.get_job(jobId)
    print("Error: {}\nError Message: {}".format(details['error'], details['error_message']))

Job 6b085cc0-c01d-4e96-8c72-f7ce13033d3f terminated with status: completed


Use the `get_result()` method to retrieve a dataframe for the SQL result set:

In [14]:
result_df = sqlClient.get_result(jobId)
print("OK, we have a dataframe for the SQL result that has been stored by SQL Query in " + sqlClient.get_job(jobId)['resultset_location'])

OK, we have a dataframe for the SQL result that has been stored by SQL Query in cos://s3.us-south.objectstorage.softlayer.net/sqltempregional/jobid=6b085cc0-c01d-4e96-8c72-f7ce13033d3f


When not needed anymore you can purge the result set from Cloud Object Storage using the `delete_result()` method:

In [15]:
sqlClient.delete_result(jobId)

,Deleted Object
0,jobid=6b085cc0-c01d-4e96-8c72-f7ce13033d3f/par...
1,jobid=6b085cc0-c01d-4e96-8c72-f7ce13033d3f/_SU...
2,jobid=6b085cc0-c01d-4e96-8c72-f7ce13033d3f


### <a id="joblist"></a> 5. List recent SQL submissions
The method `get_jobs()` provides you a dataframe with all recent SQL submissions with all details. You can change the value `-1`for `display.max_colwidth` to a positive integer if you want to truncate the cell content to shrink the overall table display size.

In [16]:
pd.set_option('display.max_colwidth', -1)
sqlClient.get_jobs().head(100)

,job_id,status,user_id,statement,resultset_location,submit_time,end_time,error,error_message
0,6b085cc0-c01d-4e96-8c72-f7ce13033d3f,completed,torsten@de.ibm.com,"SELECT o.OrderID, c.CompanyName, e.FirstName, e.LastName FROM cos://us-geo/sql/orders.parquet STORED AS PARQUET o, cos://us-geo/sql/employees.parquet STORED AS PARQUET e, cos://us-geo/sql/customers.parquet STORED AS PARQUET c WHERE e.EmployeeID = o.EmployeeID AND c.CustomerID = o.CustomerID AND o.ShippedDate > o.RequiredDate AND o.OrderDate > ""1998-01-01"" ORDER BY c.CompanyName",cos://s3.us-south.objectstorage.softlayer.net/sqltempregional/jobid=6b085cc0-c01d-4e96-8c72-f7ce13033d3f,2018-03-09T19:36:54.360Z,2018-03-09T19:37:16.992Z,None,None
1,e6fe409b-bacc-40bf-9eb0-ca9bfcce5f85,completed,torsten@de.ibm.com,"SELECT o.OrderID, c.CompanyName, e.FirstName, e.LastName FROM cos://us-geo/sql/orders.parquet STORED AS PARQUET o, cos://us-geo/sql/employees.parquet STORED AS PARQUET e, cos://us-geo/sql/customers.parquet STORED AS PARQUET c WHERE e.EmployeeID = o.EmployeeID AND c.CustomerID = o.CustomerID AND o.ShippedDate > o.RequiredDate AND o.OrderDate > ""1998-01-01"" ORDER BY c.CompanyName",cos://s3.us-south.objectstorage.softlayer.net/sqltempregional/jobid=e6fe409b-bacc-40bf-9eb0-ca9bfcce5f85,2018-03-09T19:35:50.796Z,2018-03-09T19:36:16.218Z,None,None
2,a100f6d8-c622-4fbb-9de4-37a9329adddb,completed,torsten@de.ibm.com,"SELECT o.OrderID, c.CompanyName, e.FirstName, e.LastName FROM cos://us-geo/sql/orders.parquet STORED AS PARQUET o, cos://us-geo/sql/employees.parquet STORED AS PARQUET e, cos://us-geo/sql/customers.parquet STORED AS PARQUET c WHERE e.EmployeeID = o.EmployeeID AND c.CustomerID = o.CustomerID AND o.ShippedDate > o.RequiredDate AND o.OrderDate > ""1998-01-01"" ORDER BY c.CompanyName",cos://s3-api.us-geo.objectstorage.softlayer.net/samplenotebooksbaa5120fe81044f48b0f7ed580cc5554/jobid=a100f6d8-c622-4fbb-9de4-37a9329adddb,2018-03-09T19:21:01.552Z,2018-03-09T19:21:17.516Z,None,None
3,b45bc451-9b37-483f-83c6-0133b7a5e786,completed,torsten@de.ibm.com,"SELECT o.OrderID, c.CompanyName, e.FirstName, e.LastName FROM cos://us-geo/sql/orders.parquet STORED AS PARQUET o, cos://us-geo/sql/employees.parquet STORED AS PARQUET e, cos://us-geo/sql/customers.parquet STORED AS PARQUET c WHERE e.EmployeeID = o.EmployeeID AND c.CustomerID = o.CustomerID AND o.ShippedDate > o.RequiredDate AND o.OrderDate > ""1998-01-01"" ORDER BY c.CompanyName",cos://s3-api.us-geo.objectstorage.softlayer.net/samplenotebooksbaa5120fe81044f48b0f7ed580cc5554/jobid=b45bc451-9b37-483f-83c6-0133b7a5e786,2018-03-09T19:19:29.835Z,2018-03-09T19:19:47.744Z,None,None
4,2729dffc-e6f7-47c2-8653-5e33d95f0a4e,failed,torsten@de.ibm.com,"SELECT o.OrderID, c.CompanyName, e.FirstName, e.LastName FROM cos://us-geo/sql/orders.parquet STORED AS PARQUET o, cos://us-geo/sql/employees.parquet STORED AS PARQUET e, cos://us-geo/sql/customers.parquet STORED AS PARQUET c WHERE e.EmployeeID = o.EmployeeID AND c.CustomerID = o.CustomerID AND o.ShippedDate > o.RequiredDate AND o.OrderDate > ""1998-01-01"" ORDER BY c.CompanyName",cos://s3-api.us-geo.objectstorage.softlayer.net/samplenotebooksbaa5120fe81044f48b0f7ed580cc555/jobid=2729dffc-e6f7-47c2-8653-5e33d95f0a4e,2018-03-09T19:17:28.788Z,2018-03-09T19:18:01.640Z,SQL execution failed,The specified object does not exist or the object name contains an invalid character. Specify a valid Cloud Object Storage object location.
5,3c1caba8-11ec-4dd9-aba3-e87b1d125c14,failed,torsten@de.ibm.com,"SELECT o.OrderID, c.CompanyName, e.FirstName, e.LastName FROM cos://us-geo/sql/orders.parquet STORED AS PARQUET o, cos://us-geo/sql/employees.parquet STORED AS PARQUET e, cos://us-geo/sql/customers.parquet STORED AS PARQUET c WHERE e.EmployeeID = o.EmployeeID AND c.CustomerID = o.CustomerID AND o.ShippedDate > o.RequiredDate AND o.OrderDate > ""1998-01-01"" ORDER BY c.CompanyName",cos://s3-api.us-geo.objectstorage.softlayer.net/samplenotebooksbaa5120fe81044f48b0f7ed580cc555/jobid=3c1caba8-11ec-4dd

### <a id="next"></a> 6. Next steps
You have learned in this notebook how you can use the `ibmcloudsql` library in a Python notebook to submit SQL queries on data in IBM Cloud Object Storage and how you can interact with the query results. You may want to desire now to automate such SQL query execution as part of your cloud solution. One way to achieve this is by using the <a href="https://console.bluemix.net/openwhisk/" target="_blank">IBM Cloud Functions</a> framework. There is also a SQL function available that lets you set up a cloud function to run SQL statements with IBM Cloud SQL Query. You can find the documentation for doing this <a href="https://hub.docker.com/r/ibmfunctions/sqlquery/" target="_blank" rel="noopener noreferrer">here</a>.

### <a id="authors"></a>Authors

**Torsten Steinbach**, Torsten is the lead architect for IBM Cloud SQL Query. Previously he has worked as IBM architect for a series of data management products and services, including DB2, PureData for Analytics and Db2 on Cloud.

<hr>
Copyright &copy; IBM Corp. 2018. This notebook and its source code are released under the terms of the MIT License.